# Exploratory Data Analysis (EDA)

El objetivo de este notebook es optimizar el rendimiento de la API y mejorar el rendimiento predictivo del modelo de Machine Learning para nuestro sistema de recomendación, con el fin de reducir las necesidades computacionales o de datos y mejorar la interpretabilidad de los resultados.

Para ello, haremos uso de la ingeniería de características, que consiste en crear, seleccionar o transformar las variables que se usarán para los endpoints que se consumirán en la API y el modelo.

Algunas de las tareas que realizaremos son:

- Determinar qué características son las más importantes.

- Aplicar un análisis de sentimiento con NLP.

- Codificar categóricos de alta cardinalidad.

- Reducir la dimensionalidad de los datos aplicando análisis de componentes principales para conservar la mayor parte de la información útil.

- Obtener una base de datos única y más eficiente para cada endpoint.

## 0 Configuraciones Globales e Importaciones

En esta sección, importamos todas las bibliotecas y/o modulos necesarios para nuestro proceso de feature engineering y establecemos configuraciones globales de ser requerido.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import scipy as sp
import textblob
import sklearn
from textblob import TextBlob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

print(f"System version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"scipy version: {sp.__version__}")
print(f"textblob version: {textblob.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")